In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd 
import tensorflow as tf

In [2]:
from sklearn.preprocessing import OneHotEncoder

### ResNet Architecture

In [3]:
# Residual Block Layer definition

class ResidualBlock(tf.keras.layers.Layer):

  def __init__(self, filters, strides=1, **kwargs):
    super().__init__(**kwargs)

    self.main_layers = [
        tf.keras.layers.Conv2D(filters=filters, kernel_size=3, strides=strides, padding='same'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Activation('relu'),
        tf.keras.layers.Conv2D(filters=filters, kernel_size=3, strides=1, padding='same'),
        tf.keras.layers.BatchNormalization()
    ]

    self.skip_layers = []
    if strides > 1:
      self.skip_layers = [
          tf.keras.layers.Conv2D(filters=filters, kernel_size=1, strides=strides, padding='same'),
          tf.keras.layers.BatchNormalization()
      ]

  def call(self, inputs):
    Z = inputs
    for layer in self.main_layers:
      Z = layer(Z)

    skip_Z = inputs
    for layer in self.skip_layers:
      skip_Z = layer(skip_Z)

    return tf.keras.layers.Activation('relu')(Z + skip_Z)

### Data Preprocessing

In [4]:
x_data = np.load("Dataset\X_N70000_lpx50_gqtwz.npz")["arr_0"]
y_data = np.load("Dataset\Y_N70000_lpx50_gqtwz.npz")["arr_0"][:,0] # took only the feature about the class of the particle, it cuts the mass

In [5]:
x_data = x_data.reshape(x_data.shape[0], 50, 50, 1)

# data split
num_images = x_data.shape[0]

train_end = int(0.6 * num_images)
test_end = train_end + int(0.2 * num_images)

x_train, x_test, x_val = x_data[:train_end], x_data[train_end:test_end], x_data[test_end:]
y_train, y_test, y_val = y_data[:train_end], y_data[train_end:test_end], y_data[test_end:]

one_hot_encoder = OneHotEncoder(sparse_output=False)
y_train_onehot = one_hot_encoder.fit_transform(y_train.reshape((-1,1)))
y_val_onehot = one_hot_encoder.fit_transform(y_val.reshape((-1,1)))
y_test_onehot = one_hot_encoder.fit_transform(y_test.reshape((-1,1)))


### Network Model

In [6]:
ResNet18 = tf.keras.Sequential([

    tf.keras.layers.Input(shape=[50, 50, 1]),
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D(),

    ResidualBlock(filters=64),
    ResidualBlock(filters=64),
    
    ResidualBlock(filters=128, strides=2),
    ResidualBlock(filters=128),
    
    ResidualBlock(filters=256, strides=2),
    ResidualBlock(filters=256),
    
    ResidualBlock(filters=512, strides=2),
    ResidualBlock(filters=512),

    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(5, activation='softmax')
])

In [7]:
ResNet18.compile(
    loss = tf.keras.losses.CategoricalCrossentropy(),
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001),
    metrics = ["accuracy"]
    )

history_resnet = ResNet18.fit(
    x_train, y_train_onehot,
    batch_size = 32,
    epochs = 20,
    validation_data = (x_val, y_val_onehot)
)

Epoch 1/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 479s 358ms/step - accuracy: 0.5429 - loss: 1.1873 - val_accuracy: 0.1996 - val_loss: 32.2487
Epoch 2/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 483s 368ms/step - accuracy: 0.6412 - loss: 0.9693 - val_accuracy: 0.2024 - val_loss: 7.8953
Epoch 3/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 480s 366ms/step - accuracy: 0.6594 - loss: 0.9347 - val_accuracy: 0.1996 - val_loss: 63.8013
Epoch 4/20
1313/1313 ━━━━━━━━━━━━━━━━━━━━ 489s 372ms/step - accuracy: 0.6716 - loss: 0.8982 - val_accuracy: 0.1996 - val_loss: 177.6199
Epoch 5/20
 173/1313 ━━━━━━━━━━━━━━━━━━━━ 7:01 370ms/step - accuracy: 0.6778 - loss: 0.8749

KeyboardInterrupt: 